In [43]:
import pandas as pd
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# --- Load data from System A and B (JSON), and Excel Sheet ---
def load_data():
    with open("crm_data.json") as f:
        system_a = json.load(f)

    with open("financial_data.json") as f:
        system_b = json.load(f)

    excel_data = pd.read_excel("excel_data.xlsx")  

    df_a = pd.DataFrame(system_a)
    df_b = pd.DataFrame(system_b)

    combined_df = pd.concat([df_a, df_b, excel_data], ignore_index=True)
    return combined_df

# --- Validate basic assumptions ---
def validate_data(df):
    df['company_name'] = df['CompanyName'].str.lower().str.strip()
    df['company_name'] = df['CompanyName'].str.replace(r'\b(ltd|pvt|inc|corp|pte|private|limited)\b', '', regex=True)
    df['company_name'] = df['CompanyName'].str.replace(r'\s+', ' ', regex=True)
    return df

# --- Apply fuzzy matching and remove duplicates ---
def fuzzy_deduplicate(df):
    seen = set()
    final_records = []

    for name in df['company_name']:
        match_found = False
        for seen_name in seen:
            if fuzz.token_sort_ratio(name, seen_name) > 90:
                match_found = True
                break
        if not match_found:
            seen.add(name)
            final_records.append(name)

    return pd.DataFrame({'company_name': list(seen)})

# --- Apply mapping from mapping_file ---
def apply_mapping(df, mapping_df):
    reference_list = mapping_df['reference_name'].dropna().astype(str).tolist()

    def fuzzy_match(name):
        if pd.isnull(name):
            return None
        match = process.extractOne(str(name), reference_list)
        return match[0] if match else None

    df['mapped_name'] = df['company_name'].apply(fuzzy_match)
    return df
    
# --- Main ETL Process ---
def main():
    print("Loading raw data...")
    raw_data = load_data()

    print("Validating and cleaning...")
    staging_data = validate_data(raw_data)

    print("Performing fuzzy matching and deduplication...")
    deduped_data = fuzzy_deduplicate(staging_data)

    print("Loading mapping file...")
    mapping_df = pd.read_excel("mapping_file.xlsx")

    print("Applying name mappings...")
    final_data = apply_mapping(deduped_data, mapping_df)

    print("Loading to Master Reference Table...")
    final_data.to_excel("master_reference_table.xlsx", index=False)

    print("Ready for analytics...")
    print(final_data.head())

if __name__ == "__main__":
    main()


Loading raw data...
Validating and cleaning...
Performing fuzzy matching and deduplication...
Loading mapping file...
Applying name mappings...
Loading to Master Reference Table...
Ready for analytics...
              company_name                  mapped_name
0         Quantum Dyn Ltd.   EverGreen Technologies Ltd
1  Sunrise Innovations Pte  Sunrise Innovations Pte Ltd
2      Sunrise Innovations  Sunrise Innovations Pte Ltd
3             ACME Pte Ltd                 ACME Pte Ltd
4       EverGreen Tech Ltd                 ACME Pte Ltd


In [5]:
import pandas as pd
import json
from fuzzywuzzy import fuzz, process

# File paths
crm_path = "crm_data.json"
financial_path = "financial_data.json"
excel_path = "excel_data.xlsx"
master_ref_path = "master_reference_table.xlsx"

# Step 1: Load Data
with open(crm_path) as f:
    crm_data = json.load(f)
with open(financial_path) as f:
    financial_data = json.load(f)
excel_data = pd.read_excel(excel_path)

# Step 2: Convert  data to DataFrames
crm_df = pd.DataFrame(crm_data)
financial_df = pd.DataFrame(financial_data)
excel_df = pd.DataFrame(excel_data)

# Step 3: Combine all data sources
combined_df = pd.concat([crm_df, financial_df, excel_df], ignore_index=True)

# Step 4: Clean company names in both datasets
combined_df["company_name_cleaned"] = (
    combined_df["company_name"]
    .str.lower().str.replace(",", "").str.strip()
    .str.replace(r'\b(ltd|pvt|inc|corp|pte|private|limited)\b', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
)

crm_df["company_name_cleaned"] = (
    crm_df["company_name"]
    .str.lower().str.replace(",", "").str.strip()
    .str.replace(r'\b(ltd|pvt|inc|corp|pte|private|limited)\b', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
)

# Step 5: Perform fuzzy matching against CRM data
matches = []
for _, row in combined_df.iterrows():
    result = process.extractOne(
        row["company_name_cleaned"],
        crm_df["company_name_cleaned"],
        scorer=fuzz.token_sort_ratio
    )
    if result:
        match_name, score, index = result  # ✅ Correct unpacking
        matched_row = crm_df.iloc[index]  # Directly access via index

        matched_row = crm_df[crm_df["company_name_cleaned"] == match_name].iloc[0]
        matches.append({
            "source_system": row["system_name"],
            "original_company_id": row["system_id"],
            "original_company_name": row["company_name"],
            "matched_crm_id": matched_row["system_id"],
            "matched_crm_name": matched_row["company_name"],
            "confidence_score": score
        })

# Step 6: Create and validate output
matched_df = pd.DataFrame(matches)
validated_output = matched_df[matched_df["confidence_score"] >= 85]

# Step 7: Create master reference table
master_reference_table = validated_output.copy()
master_reference_table["master_id"] = (
    master_reference_table["matched_crm_id"].astype(str) + "_" +
    master_reference_table["original_company_id"].astype(str)
)

# Save results
master_reference_table.to_excel(master_ref_path, index=False)
print(f"Master reference table saved to {master_ref_path}")


Master reference table saved to master_reference_table.xlsx
